In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# 查看gpu是否可用
device =torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork()
model.to(device)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [4]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [5]:
loss_fn = nn.CrossEntropyLoss()

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

在dataloader划分batch后，才能将tensor放入cuda；
对batch的to(device)的操作不是inplace的，所以需要重新赋值；

In [9]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)
        y = y.to(device)
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # if batch % 500 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} ")

In [ ]:
len(train_dataloader.dataset)

In [10]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
Test Error: Accuracy: 72.2%, Avg loss: 0.759266 
Epoch 2
-------------------------------
Test Error: Accuracy: 73.3%, Avg loss: 0.737076 
Epoch 3
-------------------------------
Test Error: Accuracy: 74.1%, Avg loss: 0.717318 
Epoch 4
-------------------------------
Test Error: Accuracy: 75.1%, Avg loss: 0.699408 
Epoch 5
-------------------------------
Test Error: Accuracy: 76.0%, Avg loss: 0.683057 
Epoch 6
-------------------------------
Test Error: Accuracy: 76.6%, Avg loss: 0.668095 
Epoch 7
-------------------------------
Test Error: Accuracy: 77.2%, Avg loss: 0.654360 
Epoch 8
-------------------------------
Test Error: Accuracy: 77.8%, Avg loss: 0.641732 
Epoch 9
-------------------------------
Test Error: Accuracy: 78.1%, Avg loss: 0.630122 
Epoch 10
-------------------------------
Test Error: Accuracy: 78.5%, Avg loss: 0.619440 
Epoch 11
-------------------------------
Test Error: Accuracy: 78.8%, Avg loss: 0.609609 
Epoch 12
----------

In [11]:
torch.save(model.state_dict(), "FashionMNIST.pth")

In [12]:
new_model = NeuralNetwork()
new_model.load_state_dict(torch.load("FashionMNIST.pth"))
new_model.eval()

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [14]:
test_loop(test_dataloader, new_model.to(device), loss_fn)

Test Error: Accuracy: 80.8%, Avg loss: 0.548447 


In [15]:
torch.save(model,"total_model.pth")
total_model = torch.load("total_model.pth")
total_model.eval()

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [16]:
test_loop(test_dataloader, total_model, loss_fn)

Test Error: Accuracy: 80.8%, Avg loss: 0.548447 
